Authors: Hoon Na and Sudhanshu Manda

In [1]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
from matplotlib.dates import DateFormatter
import sys
import pandas as pd
import psycopg2

In [14]:
# connect to the PostGre SQL database on AWS
engine = psycopg2.connect(
    database="cse6242team22db",
    user="Team22",
    password="vacayaway",
    host="cse6242-db.cvdup1gpakos.us-east-2.rds.amazonaws.com",
    port='5432'
)
# create table crime_data (
# 	id int primary key,
# 	report_date date,
# 	latitude float,
# 	longitude float,
# 	ucr int,
# 	ucr_literal varchar,
# 	crime_harm_index int)


# create table homeaway_listings (
# 	id int primary key,
# 	listingId varchar,
# 	price_amt float,
# 	price_avg_nightly float,
# 	price_fees float,
# 	price_rent float,
# 	latitude float,
# 	longitude float,
# 	city varchar,
# 	state varchar,
# 	country varchar,
# 	review_count int,
# 	review_average float,
# 	bathrooms int,
# 	bedrooms int,
# 	listing_url varchar,
# 	book_with_confidence boolean)

# create table yelp_data (
#     id int primary key,
#     name varchar,
#     rating float,
#     review_count int,
#     categories varchar,
#     other_id varchar,
#     location varchar,
#     price varchar,
#     latitude float,
#     longitude float,
#


cur = engine.cursor()
cur2 = engine.cursor()

# cur.execute("SELECT * FROM crime_data limit 10;")
# print(cur.fetchone())
# cur.execute("SELECT * FROM yelp_data limit 10;")
# print(cur.fetchone())
##cur.execute("SELECT * FROM homeaway_listings limit 10;")
#print(cur.fetchall())

str_category = 'Night Life'
str_subcategory = 'Bars'

# Construct SQL command

str_cmd_block1 = """SELECT rank_filter.* FROM (
SELECT hl.index,hl.listing_id,hl.review_average ,hl.price_rent, hl.bathrooms, yd.name, yd.rating, yd.primary_categories,yd.secondary_categories, yd.price, yd.categories, hl.crime_score, st_distance(hl.geo_point::geography, yd.geo_point::geography) as distance, 
rank() OVER (
PARTITION BY hl.index
ORDER BY st_distance(hl.geo_point::geography, yd.geo_point::geography)
)
FROM yelp_data yd, homeaway_v2 hl
WHERE primary_categories Like CONCAT('%',"""

str_cmd_block2 = '\'' + str_category + '\''

str_cmd_block3 = """,'%') and secondary_categories Like CONCAT('%', """

str_cmd_block4 = '\'' + str_subcategory + '\''

str_cmd_block5 = """, '%') and yd.rating > 2.5
) rank_filter WHERE RANK <= 5;"""

str_cmd = str_cmd_block1 + str_cmd_block2 + str_cmd_block3 + str_cmd_block4 + str_cmd_block5

cur.execute(str_cmd)

#primary_categories and secondary_categories
#cur.execute("select * from homeaway_listings hl join crime_data cd on st_distance(hl.geo_point::geography, cd.geo_point::geography) < 300 where hl.id =3;")
response = cur.fetchall()
temp_df = pd.DataFrame(response)
#crime_tot = pd.DataFrame()
temp_df
##crime_tot = pd.DataFrame()

str_cmd_2 = """SELECT id,listingId,review_average ,price_rent, bathrooms, crime_score
FROM homeaway_listings
ORDER BY id"""

cur2.execute(str_cmd_2)

response2 = cur2.fetchall()
temp_df_test_2 = pd.DataFrame(response2)


In [15]:
temp_df_test_2

,0,1,2,3,4,5
0,0,7512702,0.000000,1150.00,1,16.494111
1,1,7554619,5.000000,1520.00,1,18.891851
2,2,v625088,4.789473,3464.76,2,18.786659
3,3,v1424850,4.750000,1750.00,2,2.058508
4,4,7130050,4.800000,918.00,1,24.915042
5,5,7172028,4.545454,1948.00,2,28.516912
6,6,v382033,4.647059,1520.00,1,47.590545
7,7,v468017,4.428571,1098.00,1,32.681038
8,8,7263368,4.000000,1920.00,2,4.575198
9,9,v1376109,5.000000,1648.00,1,6.963036


In [ ]:
temp_df

**Data Amalgamation**

At this stage, the combined dataset contains 5 entries for each listing corresponding to the 5 nearest businesses (in order) that fall under the target User Category. 

A Distance Score (D) will be calculated for each group of entries corresponding to listing $x$ based on the following formula:

$D_x$ = Distance Score for listing $x$

$d_i$ = distance value for business listing $i$

$$D_x = \sum_{i=1}^5 \frac{i}{d_i}$$

This Distance Score formula is effective at discriminating between listings with farther away UC businesses from closer businesses. For example, a listing with UC businesses closer to it will have a higher Distance Score than a listing with UC businsses farther away.

In [16]:
# Rename columns for readability
temp_df = temp_df.rename(index=str, columns={
    0: "id",
    1: "listingId",
    2: "review_average",
    3: "price_rent",
    4: "bathrooms",
    5: "yd_name",
    6: "yd_rating",
    7: "yd_primary_categories",
    8: "yd_secondary_categories",
    9: "yd_price",
    10: "yd_categories",
    11: "crime_score",
    12: "distance",
    13: "rank"
})

temp_df_test_2 = temp_df_test_2.rename(index=str, columns={
    0: "id",
    1: "listingId",
    2: "review_average",
    3: "price_rent",
    4: "bathrooms",
    5: "crime_score",
})

In [ ]:
temp_df_test_2

In [18]:
temp_df

,id,listingId,review_average,price_rent,bathrooms,yd_name,yd_rating,yd_primary_categories,yd_secondary_categories,yd_price,yd_categories,crime_score,distance,rank
0,0,7554619,5.000000,1378.00,1.0,The Lawrence,4.0,"Night Life, Food","Cocktail Bars, Breakfast & Brunch, American",$$,"American (New), Cocktail Bars, Breakfast & Brunch",373,190.931083,1
1,0,7554619,5.000000,1378.00,1.0,Takorea,3.5,"Night Life, Food","Bars, Mexican, Korean",$$,"Bars, Korean, Mexican",373,264.629678,2
2,0,7554619,5.000000,1378.00,1.0,Bulldog's,4.0,Night Life,Gay Bars,$,Gay Bars,373,299.798713,3
3,0,7554619,5.000000,1378.00,1.0,The Sivas,3.5,"Night Life, Food","Mediterranean, Greek, Hookah Bars",$$,"Mediterranean, Greek, Hookah Bars",373,315.556865,4
4,0,7554619,5.000000,1378.00,1.0,Blue India,4.0,"Night Life, Food","Bars, Indian",$$,"Bars, Indian",373,322.745053,5
5,1,7512702,4.000000,1119.00,1.0,Community Smith,3.5,"Night Life, Food","Cocktail Bars, American",$$,"American (New), Cocktail Bars",1,323.560954,1
6,1,7512702,4.000000,1119.00,1.0,Pho King - Midtown,4.0,"Night Life, Food","Bars, Restaurants, South Asian",$$,"Vietnamese, Bars, Soup",1,328.803045,2
7,1,7512702,4.000000,1119.00,1.0,Midcity Cafe,4.0,"Night Life, Food","Bars, Cafe",None,"Cafes, Bars",1,334.092092,3
8,1,7512702,4.000000,1119.00,1.0,Alibi Lounge,3.0,Night Life,"Hookah Bars, Dance Clubs",None,"Hookah Bars, Dance Clubs",1,387.513852,4
9,1,7512702,4.000000,1119.00,1.0,The Garden Grille and bar,3.0,"Night Life, Food","Bars, American",None,"American (Traditional), Bars",1,406.615990,5


In [28]:
temp_df.loc[temp_df['distance'] == 0, 'distance'] = 0.01

In [29]:
temp_df[temp_df['distance'] == 0]

,id,listingId,review_average,price_rent,bathrooms,yd_name,yd_rating,yd_primary_categories,yd_secondary_categories,yd_price,yd_categories,crime_score,distance,rank


In [ ]:
test_df = pd.DataFrame(data = {
    "id": [0,0,0,1,1,1],
    "dist": [800, 1600, 2000, 900, 3000, 3100],
    "rank": [1,2,3,1,2,3]
})

test_df['weightedOne'] = test_df['dist'] / test_df['rank']
test_df['weightedTwo'] = test_df['rank'] / test_df['dist']
test_df['weightedThree'] = 100 / test_df['weightedOne']

In [ ]:
test_df

In [ ]:
test_df.groupby('id').sum()

In [ ]:
temp_df['D'] = temp_df['rank'] / temp_df['distance']

In [ ]:
temp_df

In [ ]:
temp_df_agg = temp_df.groupby('id').agg({
    'review_average': 'first',
    'price_rent': 'first',
    'bathrooms': 'first',
    'yd_rating': 'first',
    'crime_score': 'first',
    'D': 'sum'
})

Based on this work, the code will now be functionalized such that a Distance Score column for a specified User Category will be outputted and can be appended to the Listings Corpus. Specifically, only the last column `D` is required to be appended as the Distance Score for a specific User Category.

In [ ]:
test_df3 = temp_df_agg.drop(columns=['D'])

test_df3['test_D_UC1'] = temp_df_agg['D']

In [ ]:
test_df3

## Pipeline Steps

At this stage the following information is available and required for this part of the pipeline:
- HomeAway Listings Corpus with appended Crime Scores (LC-CS): stored in AWS PostGre SQL
- Primary/Secondary Category information for UC1, UC2, UC3

The Pipeline steps to add the Distance Scores for each UC are as follows:

1. Obtain the LC-CS

2. Obtain Distance Score columns for each UC (UC1-DS, UC2-DS, UC3-DS) and append them to LC-CS

### Pipeline Code for Step 0: Connect to the SQL database

In [ ]:
# connect to the PostGre SQL database on AWS
engine = psycopg2.connect(
    database="cse6242team22db",
    user="Team22",
    password="vacayaway",
    host="cse6242-db.cvdup1gpakos.us-east-2.rds.amazonaws.com",
    port='5432'
)

### Pipeline Code for Step 1: Obtain the LC-CS

In [ ]:
cur_LC_CS = engine.cursor()

str_cmd_LC_CS = """SELECT index AS id,listingId,review_average ,price_rent, bathrooms, crime_score
FROM homeaway_listings_v2
ORDER BY id"""

cur_LC_CS.execute(str_cmd_LC_CS)

response_LC_CS = cur_LC_CS.fetchall()
df_LC_CS = pd.DataFrame(response_LC_CS)

In [ ]:
# Rename columns for readability
df_LC_CS = df_LC_CS.rename(index=str, columns={
    0: "id",
    1: "listingId",
    2: "review_average",
    3: "price_rent",
    4: "bathrooms",
    5: "crime_score",
})

In [ ]:
df_LC_CS

### Pipeline Code Step 2: Obtain Distance Score columns

A function will be created that takes in a Primary/Secondary Category pair and outputs a Distance Score column in the form of a Pandas Series that can be appended to the LC-CS DataFrame

In [30]:
def calculate_UC_DS(str_category, str_subcategory):
    cur = engine.cursor()

    #str_category = 'Food'
    #str_subcategory = 'Poke'

    # Construct SQL command

    str_cmd_block1 = """
    SELECT rank_filter.* FROM (
        SELECT hl.index,
            hl.listing_id,
            hl.review_average,
            hl.price_rent,
            hl.bathrooms,
            yd.name,
            yd.rating,
            yd.primary_categories,
            yd.secondary_categories,
            yd.price,
            yd.categories,
            hl.crime_score,
            st_distance(hl.geo_point::geography, yd.geo_point::geography) as distance,
            rank() OVER (
                PARTITION BY hl.index
                ORDER BY st_distance(hl.geo_point::geography, yd.geo_point::geography)
                )
            FROM yelp_data yd, homeaway_v2 hl
            WHERE primary_categories Like CONCAT('%',"""

    str_cmd_block2 = '\'' + str_category + '\''

    str_cmd_block3 = """,'%') and secondary_categories Like CONCAT('%', """

    str_cmd_block4 = '\'' + str_subcategory + '\''

    str_cmd_block5 = """, '%') and yd.rating > 2.5
    ) rank_filter WHERE RANK <= 5;"""

    str_cmd = str_cmd_block1 + str_cmd_block2 + str_cmd_block3 + str_cmd_block4 + str_cmd_block5

    # execute SQL command and recieve output

    cur.execute(str_cmd)
    response = cur.fetchall()
    temp_df = pd.DataFrame(response)

    # Rename columns for readability
    temp_df = temp_df.rename(index=str, columns={
        0: "id",
        1: "listingId",
        2: "review_average",
        3: "price_rent",
        4: "bathrooms",
        5: "yd_name",
        6: "yd_rating",
        7: "yd_primary_categories",
        8: "yd_secondary_categories",
        9: "yd_price",
        10: "yd_categories",
        11: "crime_score",
        12: "distance",
        13: "rank"
    })
    
    temp_df.loc[temp_df['distance'] == 0, 'distance'] = 0.01 # Preprocessing 0-distance values out to prevent errors

    temp_df['D'] = temp_df['rank'] / temp_df['distance']

    temp_df_agg = temp_df.groupby('id').agg({
        'listingId': 'first',
        'review_average': 'first',
        'price_rent': 'first',
        'bathrooms': 'first',
        'yd_rating': 'first',
        'crime_score': 'first',
        'D': 'sum'
    })

    output_score = temp_df_agg['D']

    return(output_score)

In [ ]:
# testing parts of the code
cur_test = engine.cursor()

str_cmd_test = """SELECT DISTINCT primary_categories, secondary_categories FROM yelp_data ORDER BY primary_categories;"""
cur_test.execute(str_cmd_test)
pd.DataFrame(cur_test.fetchall())

In [ ]:
calculate_UC_DS('Night Life', 'Bars')

### Full Test Execution

Test User Categories (Primary / Secondary):
1. 'Food' / 'Poke'
2. 'Active Life' / 'Golf'
3. 'Night Life' / 'Bars'

In [31]:
# define test UC pairs of Primary/Secondary Categories

cat1 = 'Food'
subcat1 = 'Poke'

cat2 = 'Active Life'
subcat2 = 'Golf'

cat3 = 'Night Life'
subcat3 = 'Bars'


In [32]:
# Step 0: connect to the PostGre SQL database on AWS
engine = psycopg2.connect(
    database="cse6242team22db",
    user="Team22",
    password="vacayaway",
    host="cse6242-db.cvdup1gpakos.us-east-2.rds.amazonaws.com",
    port='5432'
)


# Step 1: Obtain the LC-CS
cur_LC_CS = engine.cursor()

str_cmd_LC_CS = """SELECT index,listing_id,review_average ,price_rent, bathrooms, crime_score
FROM homeaway_v2
ORDER BY index"""

cur_LC_CS.execute(str_cmd_LC_CS)

response_LC_CS = cur_LC_CS.fetchall()
df_LC_CS = pd.DataFrame(response_LC_CS)

# Rename columns for readability
df_LC_CS = df_LC_CS.rename(index=str, columns={
    0: "id",
    1: "listingId",
    2: "review_average",
    3: "price_rent",
    4: "bathrooms",
    5: "crime_score",
})

df_LC_CS['UC1-DS'] = list(calculate_UC_DS(cat1, subcat1))
df_LC_CS['UC2-DS'] = list(calculate_UC_DS(cat2, subcat2))
df_LC_CS['UC3-DS'] = list(calculate_UC_DS(cat3, subcat3))

In [33]:
df_LC_CS.iloc[55]

id                        55
listingId            4360925
review_average             5
price_rent           1080.08
bathrooms                  1
crime_score              132
UC1-DS            0.00632524
UC2-DS            0.00432642
UC3-DS               100.355
Name: 55, dtype: object

In [34]:
cur.execute("""ALTER TABLE homeaway_v2 ADD COLUMN geo_point geometry(Geometry,0);""")
cur.execute("""UPDATE homeaway_v2 SET geo_point = ST_POINT(longitude, latitude);""")

<cursor object at 0x00000252FD192D68; closed: 0>

In [6]:
# Step 0: connect to the PostGre SQL database on AWS
engine = psycopg2.connect(
    database="cse6242team22db",
    user="Team22",
    password="vacayaway",
    host="cse6242-db.cvdup1gpakos.us-east-2.rds.amazonaws.com",
    port='5432'
)

In [7]:
cur = engine.cursor()

In [19]:
cur.execute("""
SELECT *
FROM (
    SELECT listing_id, gid 
    FROM homeaway_v2
    JOIN neighborhoods nb ON st_contains(nb.atl_geom, st_setsrid(geo_point, 4326))
    ) A
JOIN T ON A.gid = T.gid
""")
testDF = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])

In [17]:
testDF.columns

Index(['listing_id', 'gid'], dtype='object')

In [21]:
testDF['crime_score']

0       66.307341
1       66.307341
2       66.307341
3       66.307341
4       66.307341
5       66.307341
6       66.307341
7       66.307341
8       66.307341
9       66.307341
10      43.923431
11      43.923431
12      43.923431
13      47.083021
14      47.083021
15      47.083021
16      47.083021
17      47.083021
18      47.083021
19      47.083021
20      47.083021
21      47.083021
22      47.083021
23      47.083021
24      47.083021
25      47.083021
26     132.150403
27     132.150403
28     132.150403
29     132.150403
          ...    
584     10.033564
585     10.033564
586     16.634925
587      4.333323
588     22.489291
589     22.489291
590     22.489291
591     22.489291
592     22.489291
593      5.268127
594      5.268127
595     10.722826
596      6.343345
597     62.230465
598     62.230465
599     62.230465
600     62.230465
601     62.230465
602     62.230465
603     62.230465
604     62.230465
605     62.230465
606     62.230465
607     62.230465
608      1